In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Working with Chinook database ontains information about the artists, songs, and albums from the music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. Like with our Factbook database, a schema diagram will help us understand the available columns and the structure of the data. Here's a schema diagram for the Chinook database:

![Chinook_database_schema](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg)

Looking at all those tables can be overwhelming at first, but generally speaking we will only need to think about the specific tables that have the data we require and their connections. Before we look at some specific tables, let's take a moment to understand the different parts of the schema diagram.

- Tables names are shown in bold, with the columns in each table listed below.
- Each table has one or more columns with shading, which indicates those columns are a primary key. We'll learn more about primary keys in a later lesson, but for now all you need to know is that each row's primary key must be unique.
- Relationships between tables are shown using lines between the tables. The lines indicate which columns are related. You may notice that at least one 'end' of the relationship will be a primary key.

As you work through this lesson, you may need to refer back to the schema diagram. This is normal, so don't be bothered when that happens. It can also be helpful to write a simple query to check the column names and some example data from any of the tables, for instance:

In [2]:
%%sql

SELECT *
 FROM sqlite_master


 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


## Joining Three Tables

First task is to gather information on a specific purchase. For one single purchase (`invoice_id`) we want to know, for each track purchased:

- The id of the track.
- The name of the track.
- The name of media type of the track.
- The price that the customer paid for the track.
- The quantity of the track that was purchased.

To gather this information, we will need to write a query that joins 3 tables: `invoice_line`, `track`, and `media_type`

In [3]:
%%sql
Select *
  FROM invoice_line
 LIMIT 5 

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [4]:
%%sql
SELECT *
  FROM track
 LIMIT 5

 * sqlite:///chinook.db
Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [5]:
%%sql
SELECT *
  FROM media_type
 LIMIT 5

 * sqlite:///chinook.db
Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


**Syntax for joining data from more than 2 tables** 
```
SELECT [column_names] FROM [table_name_one]
[join_type] JOIN [table_name_two] ON [join_constraint]
[join_type] JOIN [table_name_three] ON [join_constraint];
```

We will use the `invoice_line` table in our FROM clause, since it contains 3 of the 5 columns we want in our final query. Since our task involves looking for information about a specific `invoice_id`, let's choose an `invoice_id` value of 3.

In [6]:
%%sql
SELECT * from invoice_line
 WHERE invoice_id = 3

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
27,3,2516,0.99,1
28,3,2646,0.99,1


In [7]:
%%sql
-- # adding data from the track table --
SELECT *
  FROM invoice_line il
 INNER JOIN track t ON t.track_id = il.track_id
 WHERE invoice_id = 3

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1
27,3,2516,0.99,1,2516,Black Hole Sun,203,1,1,Soundgarden,320365,10425229,0.99
28,3,2646,0.99,1,2646,I Looked At You,214,1,1,"Robby Krieger, Ray Manzarek, John Densmore, Jim Morrison",142080,4663988,0.99


In [8]:
%%sql
-- # adding the 2nd JOIN to add data from media_type table --
SELECT * 
  FROM invoice_line il
 INNER JOIN track t ON t.track_id = il.track_id
 INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
 WHERE invoice_id = 3

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,track_id_1,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_1,media_type_id_1,name_1
27,3,2516,0.99,1,2516,Black Hole Sun,203,1,1,Soundgarden,320365,10425229,0.99,1,MPEG audio file
28,3,2646,0.99,1,2646,I Looked At You,214,1,1,"Robby Krieger, Ray Manzarek, John Densmore, Jim Morrison",142080,4663988,0.99,1,MPEG audio file


In [9]:
%%sql
-- # this is a single purchase that has the ID 4 --
SELECT 
    il.track_id, 
    t.name AS track_name, 
    mt.name AS track_type, 
    il.unit_price, 
    il.quantity
 FROM invoice_line il
 INNER JOIN track t ON t.track_id = il.track_id
 INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
 WHERE invoice_id = 4

 * sqlite:///chinook.db
Done.


track_id,track_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",Protected AAC audio file,0.99,1
2560,Violent Pornography,MPEG audio file,0.99,1
3336,War Pigs,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,MPEG audio file,0.99,1
1872,Attitude,MPEG audio file,0.99,1
748,Dealer,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
2514,Spoonman,MPEG audio file,0.99,1


## Joining more than 3 tables

Let's extend the query we wrote in the previous screen by adding the artist for each track.

In this case, we will need to join two new tables to our existing query:

- `artist`, which contains the artist name data that we need
- `album`, which has a column common to each of the `artist` and `track` tables which allows us to join those two tables.

**Our select clause won't actually use any of the columns from the `album` table. This is quite common when writing more complex queries because it will let you join to another table.**

I will need to add an `INNER JOIN` to the `album` table before adding an `INNER JOIN` to the `artist` table. This is because the tables need to have a connection point before they are joined. Between `artist` table and the other ones, there is no direct connection. The `album` table is solving this by joining it first to the 'track' table on their direct connection: `album.album_id = track.album_id`


In [10]:
%%sql
-- # this is a single purchase that has the ID 4 --
SELECT 
    il.track_id, 
    t.name AS track_name,
    a.name AS artist_name,
    mt.name AS track_type, 
    il.unit_price, 
    il.quantity
 FROM invoice_line il
 INNER JOIN track t ON t.track_id = il.track_id
 INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
 INNER JOIN album ON album.album_id = t.album_id
 INNER JOIN artist a ON a.artist_id = album.artist_id
 WHERE il.invoice_id = 4

 * sqlite:///chinook.db
Done.


track_id,track_name,artist_name,track_type,unit_price,quantity
3448,"Lamentations of Jeremiah, First Set \ Incipit Lamentatio",The King's Singers,Protected AAC audio file,0.99,1
2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
1872,Attitude,Metallica,MPEG audio file,0.99,1
748,Dealer,Deep Purple,MPEG audio file,0.99,1
1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
2514,Spoonman,Soundgarden,MPEG audio file,0.99,1


## Combining Multiple Joins with Subqueries

Produce a query that lists the top 10 artists, calculated by the number of times a track by that artist has been purchased.

In [11]:
%%sql
SELECT
    t.track_id,
    artist.name AS artist_name
 FROM track t
 INNER JOIN album ON album.album_id = t.album_id
 INNER JOIN artist ON artist.artist_id = album.artist_id
 ORDER BY 1 limit 

 * sqlite:///chinook.db
(sqlite3.OperationalError) incomplete input
[SQL: SELECT
    t.track_id,
    artist.name AS artist_name
 FROM track t
 INNER JOIN album ON album.album_id = t.album_id
 INNER JOIN artist ON artist.artist_id = album.artist_id
 ORDER BY 1 limit]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [12]:
%%sql
SELECT 
    il.invoice_line_id,
    il.track_id,
    ta.artist_name
  FROM invoice_line il
 INNER JOIN (
            SELECT
                t.track_id,
                artist.name AS artist_name
              FROM track t
             INNER JOIN album ON album.album_id = t.album_id
             INNER JOIN artist ON artist.artist_id = album.artist_id
             ) AS ta 
            ON ta.track_id = il.track_id
 ORDER BY 1 LIMIT 5

 * sqlite:///chinook.db
Done.


invoice_line_id,track_id,artist_name
1,1158,Guns N' Roses
2,1159,Guns N' Roses
3,1160,Guns N' Roses
4,1161,Guns N' Roses
5,1162,Guns N' Roses


At first it might look like we've done something wrong, because the artist for all rows is Guns N' Roses, but that's because the very first order in our table is a customer who purchased an entire Guns N' Roses album!

All that remains now is for us to add our `GROUP BY` clause, remove the extra columns and use `ORDER BY` and `LIMIT` clauses to select the 10 most popular artists.

In [13]:
%%sql
SELECT
    ta.artist_name artist,
    COUNT(ta.track_id) tracks_purchased
  FROM invoice_line il
 INNER JOIN (
             SELECT
                t.track_id,
                 artist.name AS artist_name
               FROM track t
              INNER JOIN album ON album.album_id = t.album_id
              INNER JOIN artist ON artist.artist_id = album.artist_id
             ) AS ta
            ON ta.track_id = il.track_id
 GROUP BY 1
 ORDER BY 2 DESC LIMIT 10

 * sqlite:///chinook.db
Done.


artist,tracks_purchased
Queen,192
Jimi Hendrix,187
Red Hot Chili Peppers,130
Nirvana,130
Pearl Jam,129
Guns N' Roses,124
AC/DC,124
Foo Fighters,121
The Rolling Stones,117
Metallica,106


Write a query that returns the top 5 albums, as calculated by the number of times a track from that album has been purchased. Your query should be sorted from most tracks purchased to least tracks purchased 

In [14]:
%%sql
 SELECT 
    t.album_id,
    album.title AS album,
    artist.name AS artist
  FROM track t
 INNER JOIN album ON album.album_id = t.album_id
 INNER JOIN artist ON artist.artist_id = album.artist_id
 ORDER BY 1 LIMIT 5

 * sqlite:///chinook.db
Done.


album_id,album,artist
1,For Those About To Rock We Salute You,AC/DC
1,For Those About To Rock We Salute You,AC/DC
1,For Those About To Rock We Salute You,AC/DC
1,For Those About To Rock We Salute You,AC/DC
1,For Those About To Rock We Salute You,AC/DC


In [15]:
%%sql
SELECT
    ta.album AS album,
    ta.artist AS artist,
    COUNT(*) AS tracks_purchased
 FROM invoice_line il
 INNER JOIN (
             SELECT 
                 t.album_id,
                 t.track_id,
                 album.title AS album,
                 artist.name AS artist
                 FROM track t
                INNER JOIN album ON album.album_id = t.album_id
                INNER JOIN artist ON artist.artist_id = album.artist_id
             ) AS ta
            ON ta.track_id = il.track_id
 GROUP BY 1
 ORDER BY 3 DESC LIMIT 5

 * sqlite:///chinook.db
Done.


album,artist,tracks_purchased
Are You Experienced?,Jimi Hendrix,187
Faceless,Godsmack,96
Mezmerize,System Of A Down,93
Get Born,JET,90
The Doors,The Doors,83


## Recursive Joins - joining a table to itself

In [16]:
%%sql
SELECT *
  FROM employee


 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [17]:
%%sql

SELECT
    (e1.first_name || " " || e1.last_name) AS employee_name,
    e1.title As employee_title,
    (e2.first_name || " " || e2.last_name) AS supervisor_name,
    e2.title AS supervisor_title
  FROM employee AS e1
 LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
 LIMIT 4

 * sqlite:///chinook.db
Done.


employee_name,employee_title,supervisor_name,supervisor_title
Andrew Adams,General Manager,None,None
Nancy Edwards,Sales Manager,Andrew Adams,General Manager
Jane Peacock,Sales Support Agent,Nancy Edwards,Sales Manager
Margaret Park,Sales Support Agent,Nancy Edwards,Sales Manager


## Pattern Matching Using `LIKE`

**Use the LIKE operator to find pattern matches. The syntax for LIKE is as follows:**


```
WHERE [column_name] LIKE "[pattern]"
```
Your pattern should be the substring you want to match for, and one or more % characters:

*
- `%Jen` - will match Jen at the end of a string, eg *Sarah-Jen*
- `Jen%` - will match Jen at the start of a string, eg *Jenny*
- `%Jen%` - will match Jen anywhere within the string, eg *Kris Jenner*

**SQLite `LIKE` is case insensitive**


In [18]:
%%sql
SELECT
    first_name,
    last_name,
    phone
  FROM customer
 WHERE first_name LIKE "%Jen%"

 * sqlite:///chinook.db
Done.


first_name,last_name,phone
Jennifer,Peterson,+1 (604) 688-2255


In [19]:
%%sql
SELECT
    first_name,
    last_name,
    phone
  FROM customer
WHERE first_name LIKE "%Belle%"

 * sqlite:///chinook.db
Done.


first_name,last_name,phone
Isabelle,Mercier,+33 03 80 73 66 99


## Revisiting `CASE`

In [20]:
%%sql
SELECT *
  FROM customer
 LIMIT 5

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [21]:
%%sql
SELECT *
  FROM invoice
 LIMIT 10

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
6,31,2017-01-10 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.9
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
10,31,2017-01-18 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98


In [22]:
%%sql
SELECT 
    c.customer_id,
    COUNT(i.invoice_id)
  FROM customer c
 INNER JOIN invoice i ON i.customer_id = c.customer_id
 GROUP BY 1

 * sqlite:///chinook.db
Done.


customer_id,COUNT(i.invoice_id)
1,13
2,11
3,9
4,9
5,18
6,12
7,9
8,7
9,10
10,12


In [23]:
%%sql
SELECT
    (c.first_name || " " || c.last_name) AS customer_name,
    COUNT(i.invoice_id) AS number_of_purchases,
    SUM(i.total) AS total_spent,
    CASE
        WHEN SUM(i.total) <= 40 THEN "small spender"
        WHEN 40 > SUM(i.total) <= 100 THEN "regular"
        WHEN SUM(i.total) > 100 THEN "big spender"
        END 
        AS customer_category
  FROM customer c
 INNER JOIN invoice i ON i.customer_ID = c.customer_id
 GROUP BY 1 ORDER BY 1

 * sqlite:///chinook.db
Done.


customer_name,number_of_purchases,total_spent,customer_category
Aaron Mitchell,8,70.28999999999999,regular
Alexandre Rocha,10,69.3,regular
Astrid Gruber,9,69.3,regular
Bjørn Hansen,9,72.27000000000001,regular
Camille Bernard,9,79.2,regular
Daan Peeters,7,60.38999999999999,regular
Dan Miller,12,95.03999999999999,regular
Diego Gutiérrez,5,39.6,small spender
Dominique Lefebvre,9,72.27,regular
Eduardo Martins,12,60.39,regular


In [24]:
%%sql
SELECT 
    p.playlist_id,
    p.name,
    COUNT(pt.track_id) as number_of_tracks
  FROM playlist p
  LEFT JOIN playlist_track pt ON pt.playlist_id = p.playlist_ID

 * sqlite:///chinook.db
Done.


playlist_id,name,number_of_tracks
1,Music,8715


In [25]:
%%sql

SELECT
    p.playlist_id,
    p.name AS playlist_name,
    COUNT(t.track_id) AS number_of_tracks,
    CAST(t.milliseconds AS FLOAT) / 1000 AS length_seconds
  FROM playlist AS p
  LEFT JOIN playlist_track ON p.playlist_id = playlist_track.playlist_id
  LEFT JOIN track t ON playlist_track.track_id = t.track_id
 GROUP BY 1

 * sqlite:///chinook.db
Done.


playlist_id,playlist_name,number_of_tracks,length_seconds
1,Music,3290,343.719
2,Movies,0,None
3,TV Shows,213,2622.25
4,Audiobooks,0,None
5,90’s Music,1477,230.619
6,Audiobooks,0,None
7,Movies,0,None
8,Music,3290,343.719
9,Music Videos,1,294.294
10,TV Shows,213,2622.25


In [26]:
%%sql

WITH playlist_info AS 
    (
        SELECT
    p.playlist_id,
    p.name AS playlist_name,
    t.name AS track_name,
    CAST(t.milliseconds AS FLOAT) / 1000 AS length_seconds
  FROM playlist AS p
  LEFT JOIN playlist_track ON p.playlist_id = playlist_track.playlist_id
  LEFT JOIN track t ON playlist_track.track_id = t.track_id
    ) 
    
SELECT * FROM playlist_info
 ORDER BY playlist_id


 * sqlite:///chinook.db
Done.


playlist_id,playlist_name,track_name,length_seconds
1,Music,For Those About To Rock (We Salute You),343.719
1,Music,Balls to the Wall,342.562
1,Music,Fast As a Shark,230.619
1,Music,Restless and Wild,252.051
1,Music,Princess of the Dawn,375.418
1,Music,Put The Finger On You,205.662
1,Music,Let's Get It Up,233.926
1,Music,Inject The Venom,210.834
1,Music,Snowballed,203.102
1,Music,Evil Walks,263.497


In [27]:
%%sql
SELECT * FROM invoice
 GROUP BY customer_id
 LIMIT 4

 * sqlite:///chinook.db
Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
16,1,2017-01-26 00:00:00,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,8.91
69,2,2017-05-09 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,13.86
31,3,2017-02-21 00:00:00,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,19.8
24,4,2017-02-10 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,11.879999999999999


In [44]:
%%sql
    SELECT 
        customer.*,
        SUM(invoice.total) as total_spend
      FROM customer
      LEFT JOIN invoice on invoice.customer_id = customer.customer_id
    GROUP BY customer.customer_id
    HAVING total_spend > 90


 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id,total_spend
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3,108.89999999999998
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3,99.99
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4,144.54000000000002
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5,128.7
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4,106.91999999999999
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5,98.01
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4,95.03999999999999
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5,91.08000000000001
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4,92.07000000000001
30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3,91.08


In [47]:
%%sql
SELECT
    c.*
  FROM invoice i
  INNER JOIN customer c ON i.customer_id = c.customer_id
  GROUP BY 1
  HAVING SUM(i.total) > 90


 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
17,Jack,Smith,Microsoft Corporation,1 Microsoft Way,Redmond,WA,USA,98052-8300,+1 (425) 882-8080,+1 (425) 882-8081,jacksmith@microsoft.com,5
20,Dan,Miller,None,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,None,dmiller@comcast.com,4
21,Kathy,Chase,None,801 W 4th Street,Reno,NV,USA,89503,+1 (775) 223-7665,None,kachase@hotmail.com,5
22,Heather,Leacock,None,120 S Orange Ave,Orlando,FL,USA,32801,+1 (407) 999-7788,None,hleacock@gmail.com,4
30,Edward,Francis,None,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,None,edfrancis@yachoo.ca,3


In [111]:
%%sql

WITH country_total_purchase AS (SELECT 
    c.customer_id,
    c.first_name,
    c.last_name,
    c.country,
    SUM(i.total) as total
  FROM customer c
 INNER JOIN invoice i ON i.customer_id = c.customer_id
 GROUP BY 1
 ORDER BY 4),

customer_max_purchase AS (
    SELECT 
        customer_id,
        first_name,
        last_name,
        country,
        MAX(total) as max_purchase
      FROM country_total_purchase
     GROUP BY 4 
     ORDER BY 4
)

SELECT
    cmp.country AS country,
    cmp.first_name || " " || cmp.last_name AS customer_name,
    ROUND(cmp.max_purchase, 2) AS total_purchased
  FROM customer_max_purchase AS cmp
 ORDER BY 1

 * sqlite:///chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.39
Brazil,Luís Gonçalves,108.9
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02
Czech Republic,František Wichterlová,144.54
Denmark,Kara Nielsen,37.62
Finland,Terhi Hämäläinen,79.2


In [117]:
%%sql

-- #another way or writing the same thing --
WITH customer_country_purchases AS 
            (
                SELECT
                    i.customer_id,
                    c.country,
                    SUM(i.total) AS total_purchases
                  FROM invoice i
                 INNER JOIN customer c ON c.customer_id = i.customer_id
                 GROUP BY 1, 2
            ),
    
    country_max_purchases AS 
            (
                SELECT
                    country,
                    MAX(total_purchases) AS max_purchases
                  FROM customer_country_purchases
                GROUP BY 1
            ),
        
    country_best_customer AS
            (
                SELECT
                    cmp.country,
                    cmp.max_purchases,
                    (SELECT
                        ccp.customer_id
                        FROM customer_country_purchases AS ccp
                    WHERE ccp.country = cmp.country AND cmp.max_purchases = ccp.total_purchases) AS customer_id
                FROM country_max_purchases AS cmp
            )
        
SELECT
    cbc.country country,
    c.first_name || " " || c.last_name customer_name,
    cbc.max_purchases AS total_purchased
  FROM customer c
 INNER JOIN country_best_customer cbc ON cbc.customer_id = c.customer_id
    ORDER BY 1 ASC

 * sqlite:///chinook.db
Done.


country,customer_name,total_purchased
Argentina,Diego Gutiérrez,39.6
Australia,Mark Taylor,81.18
Austria,Astrid Gruber,69.3
Belgium,Daan Peeters,60.38999999999999
Brazil,Luís Gonçalves,108.89999999999998
Canada,François Tremblay,99.99
Chile,Luis Rojas,97.02000000000001
Czech Republic,František Wichterlová,144.54000000000002
Denmark,Kara Nielsen,37.61999999999999
Finland,Terhi Hämäläinen,79.2
